<a href="https://colab.research.google.com/github/YuHsiH/freecodecamp_ML/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
'''
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
  '''

import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
train_dataset=pd.read_csv('train-data.tsv',sep='\t',names=['class','text'])
train_dataset

In [ ]:
valid_dataset=pd.read_csv('valid-data.tsv',sep='\t',names=['class','text'])
valid_dataset

In [ ]:
import seaborn as sns
sns.countplot(x=train_dataset['class'])
plt.show()

In [ ]:
# Find average number of tokens in all sentences
avg_words_len=round(sum([len(i.split()) for i in train_dataset['text']])/len(train_dataset['text']))
print(avg_words_len)

In [ ]:
# Finding Total no of unique words in corpus
s = set()
for sent in train_dataset['text']:
  for word in sent.split():
    s.add(word)
total_words_length=len(s)
print(total_words_length)

covert the spam and ham to 0 and 1 for training and validation dataset

In [ ]:
train_dataset['class']=(train_dataset['class']=='ham').astype(int)

In [ ]:
valid_dataset['class']=(valid_dataset['class']=='ham').astype(int)

`tf.data.Dataset.from_tensor_slices`

This function is used to create a TensorFlow dataset from a list of tensors. It takes a single argument, which can be:

A list of tensors (e.g., [tensor1, tensor2, ...])

A tuple containing multiple tensors (e.g., (tensor1, tensor2, ...))

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_dataset["text"].values, train_dataset["class"].values))
valid_ds = tf.data.Dataset.from_tensor_slices(
    (valid_dataset["text"].values, valid_dataset["class"].values))
train_ds

In [ ]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras.layers import TextVectorization

MAXTOKENS=total_words_length
OUTPUTLEN=avg_words_len

encoder = TextVectorization(
    max_tokens=MAXTOKENS,
    standardize='lower_and_strip_punctuation',
    output_mode='int',
    output_sequence_length=OUTPUTLEN
)
encoder.adapt(train_ds.map(lambda text, label: text)) #return text only



In [ ]:
#Create a model

model= tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), # remove return_sequence to be compatible with dense layer
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(
    train_ds,
    validation_data=valid_ds,
    validation_steps=30,
    epochs=10,
)

In [ ]:
def plot_graphs(h, metric):
    plt.plot(h.history[metric])
    plt.plot(h.history['val_'+metric])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text, encoder=encoder):

  prediction=0

  return prediction  # Return list with probability and class
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
